<a href="https://colab.research.google.com/github/kiplangatkorir/swahili-gpt2-distilled/blob/main/SwahiliGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


ValueError: mount failed

In [ ]:
!pip install transformers datasets accelerate


In [ ]:
train_path = "/content/drive/MyDrive/msingi1/data/train.txt"
valid_path = "/content/drive/MyDrive/msingi1/data/valid.txt"

# Read the text files
with open(train_path, "r", encoding="utf-8") as f:
    train_texts = f.readlines()

with open(valid_path, "r", encoding="utf-8") as f:
    valid_texts = f.readlines()


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict({"text": train_texts})
valid_dataset = Dataset.from_dict({"text": valid_texts})


In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True)
valid_dataset = valid_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/1354581 [00:00<?, ? examples/s]

Map:   0%|          | 0/169324 [00:00<?, ? examples/s]

In [ ]:
#Smaller GPT2 Model
from transformers import GPT2Config, GPT2LMHeadModel

student_config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=256,
    n_ctx=256,
    n_embd=384,
    n_layer=4,
    n_head=4,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

student_model = GPT2LMHeadModel(student_config)


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

output_dir = "/content/drive/MyDrive/msingi1/models/distilled-gpt2-swahili"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=5000,
    save_steps=5000,
    logging_steps=1000,
    learning_rate=5e-5,
    save_total_limit=2,
    fp16=True,
    gradient_accumulation_steps=2,
    push_to_hub=False,
    load_best_model_at_end=False,
    save_safetensors=True,
)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

trainer = Trainer(
    model=student_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-f1633e4deff5>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
!mkdir -p /content/drive/MyDrive/msingi1/models/distilled-gpt2-swahili


In [1]:
trainer.train(resume_from_checkpoint=True)

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

NameError: name 'trainer' is not defined